<pre style="text-align: right; width: 100%; font-size: 0.75em; line-height: 0.75em;">
+ ------------------------- + <br>
| 20/04/2025                | <br>
| Héctor Tablero Díaz       | <br>
| Álvaro Martínez Gamo      | <br>
+ ------------------------- + 
</pre>

# **Diffusers**

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./..')

import os

import torch
from torch import Tensor
from torch.utils.data import Subset
from torchvision import datasets
from torchvision.transforms import ToTensor

from image_gen import GenerativeModel
from image_gen.samplers import EulerMaruyama
from image_gen.diffusion import VarianceExploding, VariancePreserving, SubVariancePreserving, BaseDiffusion
from image_gen.noise import LinearNoiseSchedule

from typing import Tuple

import matplotlib.pyplot as plt
from IPython.display import HTML
from image_gen.visualization import display_images, create_evolution_widget

c:\Users\hecto\Desktop\Uni\AAUTO 3\Proyecto\examples\..\image_gen\samplers\euler_maruyama.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


The diffusers determine the way that noise is applied. The default classes are `VarianceExploding`, `VariancePreserving` and `SubVariancePreserving`, with the last 2 being Ornstein-Uhlenbeck processes.

### **Differences Between Diffusers**

We can see the outputs of the different models and compare with each other.

The first step is to get the dataset and define the global variables:

In [2]:
# Set up the common variables
epochs = 250
digit = 3

seed = 42

In [3]:
# Load the dataset
data = datasets.MNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor()
)

# Select a subset to speed up the training process
indices_digit = torch.where(data.targets == digit)[0]
data = Subset(data, indices_digit)

For convenience, we will create a function that includes initializing the model, training and generation:

In [ ]:
def diffusion_demo(diffusion_type, num_samples=16, **kwargs):
    """
    Demonstrates the diffusion process for a given diffusion type.
    """
    DIFFUSION_CODES = {
        VarianceExploding: "ve",
        VariancePreserving: "vp-lin",
        SubVariancePreserving: "svp-lin"
    }

    model = GenerativeModel(diffusion=diffusion_type, noise_schedule=LinearNoiseSchedule, sampler=EulerMaruyama)
    
    filename = f'saved_models/mnist_{digit}_{DIFFUSION_CODES[model.diffusion.__class__]}_euler_{epochs}e.pth'

    if os.path.isfile(filename):
        model.load(filename)
    else:
        model.train(data, epochs=epochs)
        # Tip: Save the models for them to be accessible through the dashboard
        model.save(filename)

    samples = model.generate(num_samples, **kwargs)
    display_images(samples)

In [ ]:
diffusion_demo(VarianceExploding, num_samples=16, seed=seed)

In [ ]:
diffusion_demo(VariancePreserving, num_samples=16, seed=seed)

In [ ]:
diffusion_demo(SubVariancePreserving, num_samples=16, seed=seed)

As it can be seen from these examples, `VarianceExploding` is the fastest to train, followed by `VariancePreserving` and lastly `SubVariancePreserving`, which requires a lot more epochs to produce good results.

### **Creating Custom Classes**

Custom diffusers can be created by inheriting from `BaseDiffusion`. The methods that must be implemented are `forward_sde` and `forward_process`.

It is currently not supported to load models with a custom diffuser, but it is highly recommended that they implement a `config` method for future updates.

In [ ]:
class CustomDiffuser(BaseDiffusion):
    def forward_sde(self, x: Tensor, t: Tensor) -> Tuple[Tensor, Tensor]:
        ...

    def forward_process(self, x0: Tensor, t: Tensor) -> Tuple[Tensor, Tensor]:
        ...